# **BB84 Quantum Key Distribution Protocol**

In this notebook, we will implement the BB84 protocol, a quantum key distribution protocol that allows two parties (Alice and Bob) to securely share a cryptographic key using quantum mechanics. The protocol leverages quantum superposition and the principle of measurement to ensure secure communication.

In [3]:
from qiskit import QuantumCircuit, transpile
from qiskit_aer import AerSimulator
import random

## Import Libraries
- **Qiskit**: A framework for quantum programming.
- We use AerSimulator for simulating the quantum circuit and execute to run it.
- plot_histogram is used for visualizing the output.


In [4]:
def generate_random_bits(length):
    """Generate a random bit string of given length."""
    return [random.randint(0, 1) for _ in range(length)]

def generate_random_bases(length):
    """Generate a random list of bases ('X' or 'Z') of given length."""
    return [random.choice(['X', 'Z']) for _ in range(length)]

## Random Bit and Basis Generation

- **Alice** creates a random sequence of bits (`0` or `1`) and bases (`X` or `Z`) to encode her key.  
  - The bits represent the data to encode.  
  - The bases determine how the qubits are prepared (in the standard basis $Z$ or the Hadamard basis $X$).


In [5]:

def encode_qubits(bits, bases):
    """Encode bits into qubits using the specified bases."""
    qubits = []
    for bit, basis in zip(bits, bases):
        qc = QuantumCircuit(1, 1)
        if basis == 'X':  # Encode in X-basis (Hadamard basis)
            qc.h(0)
        if bit == 1:
            qc.x(0)  # Apply X gate for 1
        if basis == 'X':  # Ensure qubit is in correct basis
            qc.h(0)
        qubits.append(qc)
    return qubits

## Encoding Qubits

- For each bit, **Alice** prepares a qubit:
  - If the basis is $Z$, the qubit is prepared as $\ket{0}$ or $\ket{1}$ based on the bit value.  
  - If the basis is $X$, she first applies a **Hadamard gate** to enter the superposition basis and encodes the bit accordingly.  
  - Finally, Alice ensures the qubit is in the correct basis by applying another **Hadamard gate** if needed.


In [6]:
def measure_qubits(qubits, bases):
    """Measure qubits in specified bases."""
    simulator = AerSimulator()
    measurements = []
    for qc, basis in zip(qubits, bases):
        if basis == 'X':  # Measure in X-basis
            qc.h(0)
        qc.measure(0, 0)

        compiled_circuit = transpile(qc, simulator)
        job = simulator.run(compiled_circuit)
        result = job.result()
        counts = result.get_counts()
        measurements.append(int(list(counts.keys())[0]))  # Extract the measured bit

    return measurements

## Bob's Measurement

- **Bob** generates a random sequence of bases ($X$ or $Z$) to measure Alice's qubits.  
  - If his basis matches Alice's, he measures the qubit accurately.  
  - If his basis differs, he measures in the wrong basis, leading to random outcomes.



In [16]:
def sift_key(alice_bits, bob_bits, alice_bases, bob_bases):
    """Sift the key by comparing bases."""
    sifted_key = []
    for a_base, b_base, a_bit, b_bit in zip(alice_bases, bob_bases, alice_bits, bob_bits):
        if a_base == b_base:  # Keep only if bases match
            sifted_key.append(0)
        else:
            sifted_key.append(1)            
    return sifted_key


## Key Sifting

- **Alice** and **Bob** compare their bases.  
- For every qubit where the bases match, they retain the corresponding bit.  
- This process eliminates any mismatches caused by differing bases.


In [17]:
# Step 1: Alice generates random bits and bases
key_length = 10  # Length of the key to generate
alice_bits = generate_random_bits(key_length)
alice_bases = generate_random_bases(key_length)

# Step 2: Alice encodes her bits into qubits
encoded_qubits = encode_qubits(alice_bits, alice_bases)

# Step 3: Bob generates random bases and measures Alice's qubits
bob_bases = generate_random_bases(key_length)
bob_bits = measure_qubits(encoded_qubits, bob_bases)

# Step 4: Alice and Bob compare their bases
sifted_key = sift_key(alice_bits, bob_bits, alice_bases, bob_bases)

# Output the results
print("Alice's bits:   ", alice_bits)
print("Alice's bases:  ", alice_bases)
print("Bob's bases:    ", bob_bases)
print("Bob's bits:     ", bob_bits)
print("Base:           ", sifted_key)

Alice's bits:    [0, 0, 0, 1, 1, 0, 1, 0, 1, 0]
Alice's bases:   ['X', 'Z', 'Z', 'X', 'X', 'X', 'X', 'X', 'Z', 'X']
Bob's bases:     ['X', 'X', 'X', 'Z', 'X', 'X', 'Z', 'X', 'Z', 'X']
Bob's bits:      [1, 1, 0, 0, 1, 0, 0, 0, 1, 0]
Base:            [0, 1, 1, 1, 0, 0, 1, 0, 0, 0]


## Purpose

This code demonstrates the principles of the **BB84 protocol**:  
- **Quantum Superposition and Measurement**: The encoding and measurement depend on the choice of basis.  
- **Key Agreement**: By comparing bases, Alice and Bob can securely agree on a shared key, even if an eavesdropper intercepts the qubits (not shown here).
